In [ ]:
deps_path = '/kaggle/input/czii-cryoet-dependencies'

In [ ]:
! cp -r /kaggle/input/czii-cryoet-dependencies/asciitree-0.3.3/ asciitree-0.3.3/

In [ ]:
! pip wheel asciitree-0.3.3/asciitree-0.3.3/

In [ ]:
!pip install asciitree-0.3.3-py3-none-any.whl

In [ ]:
! pip install -q --no-index --find-links {deps_path} --requirement {deps_path}/requirements.txt

In [ ]:
# pip installがされたかの確認
!pip show monai

In [ ]:
from typing import List, Tuple, Union
import numpy as np
import torch
from monai.data import DataLoader, Dataset, CacheDataset, decollate_batch
from monai.transforms import (
    Compose,
    EnsureChannelFirstd,
    Orientationd,
    AsDiscrete,
    RandFlipd,
    RandRotate90d,
    NormalizeIntensityd,
    RandCropByLabelClassesd,
)

In [ ]:
# 指定された次元を完全に覆うために最小限の重複でパッチの開始位置を計算する
def calculate_patch_starts(dimension_size: int, patch_size: int) -> List[int]:
    """
    Calculate the starting positions of patches along a single dimension
    with minimal overlap to cover the entire dimension.

    Parameters:
    -----------
    dimension_size : int
        Size of the dimension
    patch_size : int
        Size of the patch in this dimension

    Returns:
    --------
    List[int]
        List of starting positions for patches
    """
    if dimension_size <= patch_size:
        return [0]

    # Calculate number of patches needed
    n_patches = np.ceil(dimension_size / patch_size)

    # ここのコードはいらない気もするが一用残しておく
    if n_patches == 1:
        return [0]

    # Calculate overlap
    total_overlap = (n_patches * patch_size - dimension_size) / (n_patches - 1)

    # Generate starting positions
    positions = []
    for i in range(int(n_patches)):
        pos = int(i * (patch_size - total_overlap))
        if pos + patch_size > dimension_size:
            pos = dimension_size - patch_size
        if pos not in positions:  # Avoid duplicates
            positions.append(pos)

    return positions

def extract_3d_patches_minimal_overlap(arrays: List[np.ndarray], patch_size: int) -> Tuple[List[np.ndarray], List[Tuple[int, int, int]]]:
    """
    Extract 3D patches from multiple arrays with minimal overlap to cover the entire array.
    複数の3D配列から最小限の重複を持つバッチを抽出し、全体をカバーします
    
    Parameters:
    -----------
    arrays : List[np.ndarray]
        List of input arrays, each with shape (m, n, l)
        抽出する立方体パッチのサイズ(a x a)
    patch_size : int
        Size of cubic patches (a x a x a)
        抽出する立方体パッチサイズ
        
    Returns:
    --------
    patches : List[np.ndarray]
        List of all patches from all input arrays
        全ての入力配列からちゅしゅつされたパッチのリスト
    coordinates : List[Tuple[int, int, int]]
        List of starting coordinates (x, y, z) for each patch
        各パッチの開始位置
    """
    # 入力が非空のリストであることを確認
    if not arrays or not isinstance(arrays, list):
        raise ValueError("Input must be a non-empty list of arrays")

    # 全ての配列が同じ形状を持つ配列であることを確認
    # Verify all arrays have the same shape
    shape = arrays[0].shape
    if not all(arr.shape == shape for arr in arrays):
        raise ValueError("All input arrays must have the same shape")

    # パッチサイズが各次元の最小サイズより小さいことを確認
    if patch_size > min(shape):
        raise ValueError(f"patch_size ({patch_size}) must be smaller than smallest dimension {min(shape)}")
    
    m, n, l = shape
    patches = [] # 抽出されたパッチを格納するリスト
    coordinates = [] # 各パッチの開始座標を格納するリスト
    
    # Calculate starting positions for each dimension
    # 各次元に対するパッチの開始位置を計算
    x_starts = calculate_patch_starts(m, patch_size)
    y_starts = calculate_patch_starts(n, patch_size)
    z_starts = calculate_patch_starts(l, patch_size)
    
    # Extract patches from each array
    # 各配列からパッチを抽出
    for arr in arrays:
        for x in x_starts:
            for y in y_starts:
                for z in z_starts:
                    # 配列からパッチを切り出し
                    patch = arr[
                        x:x + patch_size,
                        y:y + patch_size,
                        z:z + patch_size
                    ]
                    patches.append(patch)
                    coordinates.append((x, y, z))
    
    return patches, coordinates # パッチのリストと座標のリストを返す


# 分割されたパッチとその開始座標から元の3D配列を再構築する
def reconstruct_array(patches: List[np.ndarray], 
                     coordinates: List[Tuple[int, int, int]], 
                     original_shape: Tuple[int, int, int]) -> np.ndarray:
    """
    Reconstruct array from patches.
    
    Parameters:
    -----------
    patches : List[np.ndarray]
        List of patches to reconstruct from
    coordinates : List[Tuple[int, int, int]]
        Starting coordinates for each patch
    original_shape : Tuple[int, int, int]
        Shape of the original array
        
    Returns:
    --------
    np.ndarray
        Reconstructed array
    """
    # 原始配列を再構築するためのゼロ配列を作成
    reconstructed = np.zeros(original_shape, dtype=np.int64)  # To track overlapping regions

    # パッチのサイズを取得(立方体パッチとして最初の次元のみを使用)
    patch_size = patches[0].shape[0]

    # 各パッチとその開始座標を順に処理
    for patch, (x, y, z) in zip(patches, coordinates):
        # 再構築配列の対応する位置にパッチを配置
        reconstructed[
            x:x + patch_size,
            y:y + patch_size,
            z:z + patch_size
        ] = patch # パッチの値で上書き

    # 再構築された配列を返す
    return reconstructed




In [ ]:
import pandas as pd
import numpy as np

# 辞書をデータフレームに変換
def dict_to_df(coords_dict, experiment_name):
    # Create lists to store data
    all_coords = []
    all_labels = []

    for label, coords in coords_dict.items():
        all_coords.append(coords)
        all_labels.extend([label] * len(coords))

    # Concatenate all coordinates
    # すべての座標を連結
    # .vstack()は、配列を垂直方向に連結する
    all_coords = np.vstack(all_coords)

    df = pd.DataFrame({
        'experiment' : experiment_name,
        'particle_type' : all_labels,
        'x' : all_coords[:, 0],
        'y' : all_coords[:, 1],
        'z' : all_coords[:, 2]
    })

    return df

In [ ]:
TRAIN_DATA_DIR = "/kaggle/input/create-numpy-dataset-exp-name"
TEST_DATA_DIR = "/kaggle/input/czii-cryo-et-object-identification"

In [ ]:
train_names = ['TS_5_4', 'TS_69_2', 'TS_6_6', 'TS_73_6', 'TS_86_3', 'TS_99_9']
valid_names = ['TS_6_4']

train_files = []
valid_files = []

for name in train_names:
    # 画像データとラベルデータを読み込む
    image = np.load(f"{TRAIN_DATA_DIR}/train_image_{name}.npy")
    label = np.load(f"{TRAIN_DATA_DIR}/train_label_{name}.npy")

    train_files.append({"image": image, "label": label})

for name in valid_names:
    image = np.load(f"{TRAIN_DATA_DIR}/train_image_{name}.npy")
    label = np.load(f"{TRAIN_DATA_DIR}/train_label_{name}.npy")

    valid_files.append({"image": image, "label": label})

In [ ]:
# Non-random transforms to be cached

# トランスフォームの定義
non_random_transforms = Compose([
    # チャンネル次元を先頭に配置。画像とラベルのデータに適用
    EnsureChannelFirstd(keys=["image", "label"], channel_dim="no_channel"),
    # 画像データの強度値を正規化(標準化)する
    NormalizeIntensityd(keys="image"),
    # 画像とラベルのオリエンテーションを"RAS"(右、前、上)に統一する
    Orientationd(keys=["image", "label"], axcodes="RAS")
])

# データの前処理結果をキャッシュする
raw_train_ds = CacheDataset(data=train_files, transform=non_random_transforms, cache_rate=1.0)

my_num_samples = 16
train_batch_size = 1

# Random transforms to be applied during training
# トレーニング中に適用されるランダムなトランスフォームの定義
random_transforms = Compose([
    # ラベルのクラスごとにランダムに切り取るトランスフォーム
    RandCropByLabelClassesd(
        keys=["image", "label"],
        label_key="label",
        spatial_size=[98, 98, 98], # 切り取り後の空間サイズ(深さ、高さ、幅)
        num_samples=my_num_samples # 生成するサンプル数
    ),
    # 画像およびラベルを90度単位でランダムに回転させるトランスフォーム
    RandRotate90d(keys=["image", "label"], prob=0.5, spatial_axes=[0, 2]),
    # 画像およびラベルを指定した軸に沿ってランダムに反転させるトランスフォーム
    RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=0),
])

train_ds = Dataset(data=raw_train_ds, transform=random_transforms)

# DataLoader remains the same
train_loader = DataLoader(
    train_ds, # トレーニングデータセット
    batch_size=train_batch_size, # バッチサイズ
    shuffle=True, # データをシャッフル
    num_workers=4, # 使用するワーカーの数
    pin_memory=torch.cuda.is_available() # CPUが利用可能な場合、ピンメモリを使用
)

# データローダーの確認
print(f"Number of workers: {train_loader.num_workers}")
print(f"Pin memory: {train_loader.pin_memory}")
print(f"Number of samples in raw_train_ds: {len(raw_train_ds)}")
print(f"Number of samples in train_ds: {len(train_ds)}")

In [ ]:
val_images, val_labels = [dcts['image'] for dcts in valid_files], [dcts['label'] for dcts in valid_files]

# バリデーション用の画像データとラベルデータから3Dパッチを抽出
# パッチサイズは96、重複を最小限に抑えて抽出
val_image_patches, _ = extract_3d_patches_minimal_overlap(val_images, 96)
val_label_patches, _ = extract_3d_patches_minimal_overlap(val_labels, 96)

val_patched_data = [{"image": img, "label": lbl} for img, lbl in zip(val_image_patches, val_label_patches)]

# データの前処理結果をキャッシュする
valid_ds = CacheDataset(data=val_patched_data, transform=non_random_transforms, cache_rate=1.0)

valid_batch_size = 16
# DataLoader remains the same
valid_loader = DataLoader(
    valid_ds,
    batch_size=valid_batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=torch.cuda.is_available()
)

print(f"Number of workers: {valid_loader.num_workers}")
print(f"Pin memory: {valid_loader.pin_memory}")
print(f"Number of samples in valid_ds: {len(valid_ds)}")

In [ ]:
# Initialize the model

import lightning.pytorch as pl

from monai.networks.nets import UNet
from monai.losses import TverskyLoss
from monai.metrics import DiceMetric

class Model(pl.LightningModule):
    def __init__(
        self,
        spatial_dims: int = 3, # データの空間次元数(2Dなら2、3Dなら3)
        in_channels: int = 1, # 入力データのチャンネル数 (グレースケールなら1、RGBなら3)
        out_channels: int = 7, # 出力データのチャンネル数(クラス数)
        channels: Union[Tuple[int, ...], List[int]] = (48, 64, 80, 80), # 各層のフィルター数
        strides: Union[Tuple[int, ...], List[int]] = (2, 2, 1), #各層のストライド (畳み込みの移動幅)
        num_res_units: int = 1, # 各層における残差ユニットの数
        lr: float=1e-3 # 学習率
        ):

        super().__init__() # 親クラスの初期化メソッドを呼び出す
        self.save_hyperparameters() # ハイパーパラメータを保存
        # UNetモデルのインスタンスを作成
        # self.save_hyperparameters()を呼び出すことで、__init__メソッドメソッドに渡されたハイパーパラメータが自動的にself.hparamsに保存される
        self.model = UNet(
            spatial_dims=self.hparams.spatial_dims,
            in_channels=self.hparams.in_channels,
            out_channels=self.hparams.out_channels,
            channels=self.hparams.channels,
            strides=self.hparams.strides,
            num_res_units=self.hparams.num_res_units,
        )
        # TverskyLossを損失関数として初期化する
        # include_background=True: 背景クラスも損失計算に含める
        # to_onehot_y=True: ターゲットラベルをワンホット形式に変換する
        # softmax=True: モデルの出力にソフトマックス関数を適用して確率値に変換する
        self.loss_fn = TverskyLoss(include_background=True, to_onehot_y=True, softmax=True) # softmax= True for multiclass
        # DiceMetricをメトリック関数として初期化する
        # 正確なセグメンテーションの評価に使用される
        # reduction="mean": バッチ内の各サンプルの平均を取る
        # ignore_empty=True: 対象オブジェクトがない場合評価から除外する
        self.metric_fn = DiceMetric(include_background=False, reduction="mean", ignore_empty=True)

        self.train_loss = 0 # トレーニング中の累計損失
        self.val_metric = 0 # バリデーション中の累計メトリック(評価指標)
        self.num_train_batch = 0 # トレーニングで使用したバッチ数のカウンタ
        self.num_val_batch = 0 # バリデーションで使用したバッチ数のカウンタ

    def forward(self, x):
        return self.model(x) # モデルの出力を返す

    # トレーニングステップ
    # トレーニング中に1バッチ分の処理を行う
    def training_step(self, batch, batch_idx):
        # バッチから画像とラベルを抽出する: 'image'キーから入力データxを、'label'キーからターゲットデータ(正解ラベル)yを取得
        x, y = batch['image'], batch['label']
        # self(x) を呼び出すと、定義済みの forward() メソッド経由でモデルに入力 x を渡し、
        # 予測結果 y_hat を得る（内部では self.model(x) が実行される）
        # LightningModuleの内部でcallが実装されており、self(x)と記述すると内部的にself.forward(x)が呼び出される
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y) # 予測結果y_hatと正解ラベルyの損失を計算
        self.train_loss += loss # トレーニング中の累計損失に加算
        self.num_train_batch += 1 # トレーニングで使用したバッチ数をカウント
        torch.cuda.empty_cache() # GPUの未使用キャッシュを解放して、メモリ使用効率を向上させる
        return loss

    # 1エポックのトレーニングが終了したときに呼び出される
    def on_train_epoch_end(self):
        loss_per_epoch = self.train_loss / self.num_train_batch # 1エポックの平均損失を計算
        self.log('train_loss', loss_per_epoch, prog_bar=True) # トレーニング損失をログに記録
        self.train_loss = 0
        self.num_train_batch = 0

    # バリデーションステップ
    # バリデーションデータに対して1バッチ分の評価処理を実行する
    def validation_step(self, batch, batch_idx):
        # バリデーション時は勾配を計算しないため、torch.no_grad()コンテキストを使用する
        with torch.no_grad(): # This ensures that gradients are not stored in memory
            # バッチから入力画像xと正解ラベルyを抽出する
            x, y = batch['image'], batch['label']
            y_hat = self(x)

            # decollate_batch() でバッチ内の各サンプルに分割し、AsDiscreteトランスフォームを適用して予測結果を one-hot形式に変換する
            # argmax=Trueを指定することで、クラスごとのスコアから最大値を持つクラスを選択
            metric_val_outputs = [AsDiscrete(argmax=True, to_onehot=self.hparams.out_channels)(i) for i in decollate_batch(y_hat)]
            # decollate_batch()で正解ラベルを分割し、AsDiscreteトランスフォームでone-hot形式に変換する
            metric_val_labels = [AsDiscrete(to_onehot=self.hparams.out_channels)(i) for i in decollate_batch(y)]

            # compute metric for current iteration
            # 現在のバッチに対して評価指標(DiceMetric)を計算するため、変換後の予測値とラベルを渡す
            self.metric_fn(y_pred=metric_val_outputs, y=metric_val_labels)
            # aggregate()メソッドでバッチ内の評価指標を"mean_batch"単位で集約し、1バッチ全体の平均を算出する
            metrics = self.metric_fn.aggregate(reduction="mean_batch")
            # 集約した評価指標の全体平均を計算(ここでは全クラスの平均値を求めている)
            val_metric = torch.mean(metrics) # I used mean over all particle species as the metric. This can be explored.
            # 累積バリデーション評価指標に現在の評価値を加算
            self.val_metric += val_metric
            # バリデーションで使用したバッチ数をカウント
            self.num_val_batch += 1

        torch.cuda.empty_cache()
        # 現在のバリデーションステップの評価指標を辞書形式で返す
        return {'val_metric': val_metric}

    # 1エポック毎のバリデーションプロセスの最後に実行される処理
    def on_validation_epoch_end(self):
        metric_per_epoch = self.val_metric / self.num_val_batch
        self.log('val_metric', metric_per_epoch, prog_bar=True, sync_dist=False) # sync_dist=True for distributed training
        self.val_metric = 0
        self.num_val_batch = 0

    # configure_optimizers()メソッドは、最適化アルゴリズムを定義するために使用される
    def configure_optimizers(self):
        # torch.optim.AdamWを使用してAdamWを用いてパラメータを最適化するオプティマイザを生成する
        # self.parameters()によって、このモデル内の全てのパラメータ(重みやバイアス)が最適化対象となる
        # 学習率は、self.hparams.lrで指定された値を使用する
        return torch.optim.AdamW(self.parameters(), lr=self.hparams.lr)


In [ ]:
channels = (48, 64, 80, 80)
strides_pattern = (2, 2, 1)
num_res_units = 1
learning_rate = 1e-3
num_epochs = 100

model = Model(channels=channels, strides=strides_pattern, num_res_units=num_res_units, lr=learning_rate)

In [ ]:
# PyTorchの内部でfloat32マトリックス乗算を実行する際の計算精度を'neduyn'に設定
torch.set_float32_matmul_precision('medium')

# Check if CUDA is available and then count the GPUs
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs: {num_gpus}")
else:
    print("No GPU available. Running on CPU.")
devices = list(range(num_gpus))
print(devices)

# PyTorch Lighting の Trainer オブジェクトを初期化して、トレーニングの各種設定を行う
trainer = pl.Trainer(
    max_epochs=num_epochs, # トレーニングするエポックの最大数
    #strategy="ddp_notebook",
    accelerator="gpu", # トレーニングでGPUを使用することを指定
    devices=[0], # 使用するGPUのインデックスをリスト形式で指定(ここでは0番目のGPUを使用)
    num_nodes=1, # 分散トレーニングを行う場合のノード数(ここでは単一ノード)
    log_every_n_steps=10, # 10ステップ毎にログ情報を出力
    enable_progress_bar=True, # 進捗バーを表示
)

In [ ]:
trainer.fit(model, train_loader, valid_loader)